# Sample Base area estimation - design

## 0 - initialisation steps

### 0.1 - Import libs

In [ ]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geopandas as gpd
import pandas as pd 
from sepal_ui.mapping import SepalMap

import helpers as h

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

### 0.1 - Basic inputs

In [ ]:
country = "kenya" # country case

grid_size = 0.5 # Grid Size (in degrees)

# Directory where output and temp files will go
# relative path from home
outdir = f"sbae_design/{country.lower()}/"

## 1 - Generate the grid 

### 1.1 - Generate the grid points

In [ ]:
# Generate the grid
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', country.capitalize()))

grid = h.generate_grid(aoi, grid_size, grid_size, 0, 0)[1]

# get a random point over each grid cell
randomInCell = grid.map(h.getRandomPoint)

# get the centroid of each gridcell
centerInCell = grid.map(h.getCenterPoint)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 5,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.setId)

### 1.2 - Display in a map

In [ ]:
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, f"aoi: {country}")
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, "gridcells")
Map.addLayer(centerInCell.draw(color="white", pointRadius=1), {}, '1 Center Sample per gridcell')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=1), {}, '1 random Sample per gridcell')

Map

### 1.3 add auxiliary data from global datasets 

In [ ]:
## Global Forest Change (Hansen et al., 2013)
gfc_col = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').select(['treecover2000','loss','lossyear','gain'],['gfc_tc00','gfc_loss','gfc_year','gfc_gain'])

## ESA WorldCover 2020
esa_20  = ee.Image('ESA/WorldCover/v100/2020').rename('esa_lc20')

## Tropical Moist Forest - JRC 2021
tmf_annual= ee.ImageCollection('projects/JRC/TMF/v1_2020/AnnualChanges').mosaic()
tmf_annual_n = tmf_annual.rename(tmf_annual.bandNames().map(h.rename_TMF))

tmf_subtp = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_Subtypes').mosaic().rename('tmf_subtypes')
tmf_main  = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').mosaic().rename('tmf_main_cl')
tmf_deg   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DegradationYear').mosaic().rename('tmf_deg_yr')
tmf_def   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').mosaic().rename('tmf_def_yr')

##  COMBINE COLLECTIONS
glo_ds = esa_20.addBands(gfc_col).addBands(tmf_subtp).addBands(tmf_main).addBands(tmf_deg).addBands(tmf_def).addBands(tmf_annual_n)

## EXTRACT THE DATA TO THE POINTS
columns = [
    'esa_lc20','gfc_tc00','gfc_loss','gfc_year','gfc_gain', 'tmf_main_cl','tmf_subtypes','tmf_1990','tmf_1995',
    'tmf_2000','tmf_2005','tmf_2010','tmf_2015','tmf_2020','tmf_def_yr','tmf_deg_yr',
]
centerPtsAux = glo_ds.reduceRegions(**{
  "reducer": ee.Reducer.first(),
  "collection": centerInCell
}).select(['point_id',*columns, '.geo'])

### 1.4 extract as a table 

In [ ]:
json = centerPtsAux.getInfo()

In [ ]:
## MAKE AS A GEODATAFRAME AND EXPORT
gdf = gpd.GeoDataFrame.from_features(json)
gdf = gdf.join(gdf["geometry"].apply(lambda p: list(p.coords)).explode().apply(pd.Series).rename(columns=({0:"LON", 1:"LAT"})))
gdf['PLOTID']=gdf['point_id']
gdf = gdf[['LON', 'LAT', 'PLOTID',*columns]]
gdf.PLOTID = gdf.PLOTID.astype(str).astype(int)
gdf = gdf.sort_values("PLOTID", ascending=True)
gdf

In [ ]:
## Export
folder = Path.home()/outdir
folder.mkdir(exist_ok=True, parents=True)
gdf.to_csv(folder/"points_ceo.csv", index=False)